In [21]:
import ast
import fastapi
import numpy as np
import pandas as pd
import pyarrow.parquet as pq
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
app = fastapi.FastAPI()

model_data = pd.read_csv("data/model_data.csv", index_col=0, parse_dates=["date"])
model_data['popular_genres'].fillna('', inplace=True)
model_data['common_genres'].fillna('', inplace=True)
model_data['unpopular_genres'].fillna('', inplace=True)

@app.get("/")
def test():
    return {"API STEAM GAMES ACTIVE"}

In [16]:
@app.get("/recommend/{title}")
def recommend_similar_games(title):

    # Encuentra el índice del juego ingresado
    game_index = model_data[model_data['title'] == title].index[0]

    # Filtra los juegos basados en la fecha de lanzamiento
    year = model_data["date"].dt.year[game_index]
    five_years_ago = year - 5
    five_years_later = year + 5
    filtered_data = model_data[(model_data["date"].dt.year >= five_years_ago) & (model_data["date"].dt.year <= five_years_later)]

    # Filtra los juegos basados price category
    price = model_data["price_discr"][game_index]
    upper_price = price + 1
    lower_price = price - 1
    filtered_data = filtered_data[(filtered_data["price_discr"] >= lower_price) & (filtered_data["price_discr"] <= upper_price)]
    
    filtered_data = filtered_data.reset_index(drop=True)
    new_game_index = filtered_data[filtered_data['title'] == title].index[0]

    # Create similarity matrix based on popular genres for filtered_data
    popular_genres_matrix = vectorizer.fit_transform(filtered_data['popular_genres'])
    popular_genres_similarity_matrix = cosine_similarity(popular_genres_matrix, popular_genres_matrix)
    # Create similarity matrix based on common genres for filtered_data
    common_genres_matrix = vectorizer.fit_transform(filtered_data['common_genres'])
    common_genres_similarity_matrix = cosine_similarity(common_genres_matrix, common_genres_matrix)
    # Create similarity matrix based on unpopular genres for filtered_data
    unpopular_genres_matrix = vectorizer.fit_transform(filtered_data['unpopular_genres'])
    unpopular_genres_similarity_matrix = cosine_similarity(unpopular_genres_matrix, unpopular_genres_matrix)
    # Combine the similarity matrices for the three genre categories
    similarity_matrix = popular_genres_similarity_matrix + common_genres_similarity_matrix + unpopular_genres_similarity_matrix

    # Adjust the game indices for the filtered_data
    similar_game_indices = similarity_matrix[new_game_index].argsort()[::-1][1:6]

    # Obtain the names of similar games and their similarity scores
    similar_game_scores = similarity_matrix[game_index][similar_game_indices]
    similar_game_titles = filtered_data.iloc[similar_game_indices]['title'].tolist()

    # Create a DataFrame with titles and similarity scores
    similar_games_dict = {'recommendations': similar_game_titles}

    return similar_games_dict

recommend_similar_games("Alter Ego")

{'recommendations': ['Zork Anthology',
  'The Castles of Dr. Creep',
  'Wasteland 1 - The Original Classic',
  'Prehistorik',
  'Dragon Wars']}

In [ ]:
@app.get("/users_recommend/{year}") # Devuelve el top 3 de juegos MÁS recomendados por usuarios para el año dado.
def UsersRecommend(year: int):

    year_reviews = australian_users_reviews[["item_id", "sentiment", "recommend"]][(pd.to_datetime(australian_users_reviews['date']).dt.year == year) & (australian_users_reviews['sentiment'] >= 1)]
    top_3_games = year_reviews["item_id"].value_counts().head(3)
    top_3_games = pd.DataFrame({'item_id': top_3_games.index, 'count': top_3_games.values})
    top_3_games = pd.merge(left=top_3_games, right=steam_games[['item_id', 'title']], on='item_id', how='left')
    most_recommended_games = [{"Puesto 1": top_3_games["title"][0]}, {"Puesto 2": top_3_games["title"][1]}, {"Puesto 3": top_3_games["title"][2]}]
    return most_recommended_games

UsersRecommend(2014)

[{'Puesto 1': 'Team Fortress 2'},
 {'Puesto 2': 'Counter-Strike: Global Offensive'},
 {'Puesto 3': "Garry's Mod"}]

In [ ]:
@app.get("/users_not_recommend/{year}") # Devuelve el top 3 de juegos MENOS recomendados por usuarios para el año dado.
def UsersNotRecommend(year: int):
    year_reviews = australian_users_reviews[["item_id", "sentiment", "recommend"]][(pd.to_datetime(australian_users_reviews['date']).dt.year == year) & (australian_users_reviews['sentiment'] >= 1)]
    bot_3_games = year_reviews["item_id"].value_counts().tail(3)
    bot_3_games = pd.DataFrame({'item_id': bot_3_games.index, 'count': bot_3_games.values})
    bot_3_games = pd.merge(left=bot_3_games, right=steam_games[['item_id', 'title']], on='item_id', how='left')
    least_recommended_games = [{"Puesto 1": bot_3_games["title"][0]}, {"Puesto 2": bot_3_games["title"][1]}, {"Puesto 3": bot_3_games["title"][2]}]
    return least_recommended_games

UsersNotRecommend(2014)

[{'Puesto 1': 'Port of Call'},
 {'Puesto 2': 'Umbrella Corps™/Biohazard Umbrella Corps™'},
 {'Puesto 3': 'My Lands: Black Gem Hunting'}]

In [ ]:
@app.get("/sentiment_analysis/{year}") # Returns a list with the count of user review records categorized with sentiment analysis according to the given year 
def sentiment_analysis(year: int):
    year_reviews = australian_users_reviews['sentiment'][pd.to_datetime(australian_users_reviews['date']).dt.year == year]
    sentiment_count = {"Negative": 0, "Neutral": 0, "Positive": 0}
    for rate in year_reviews:
        if rate == 0:
            sentiment_count["Negative"] += 1
        elif rate == 1:
            sentiment_count["Neutral"] += 1
        elif rate == 2:
            sentiment_count["Positive"] += 1
    return sentiment_count

sentiment_analysis(2018)

{'Negative': 0, 'Neutral': 0, 'Positive': 0}